In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from datetime import date
import scipy.stats as st
import copy

In [2]:
df=pd.read_excel("/home/user/Desktop/env2/VAR/mvp_3/currency_conv.xlsx")

xls = pd.ExcelFile('/home/user/Desktop/env2/VAR/mvp_3/interest_rates.xlsx')
LIBOR_int=pd.read_excel(xls,"LIBOR")
LIBOR_int.index=LIBOR_int["Date"]

SOFR_int=pd.read_excel(xls,"SOFR")
SOFR_int.index=SOFR_int["Date"]

SONIA_int=pd.read_excel(xls,"SONIA")
SONIA_int.index=SONIA_int["Date"]

TIIE_int=pd.read_excel(xls,"TIIE")
TIIE_int.index=TIIE_int["Date"]

BSBY_int=pd.read_excel(xls,"BSBY")
BSBY_int.index=BSBY_int["Date"]
#print(TIIE_int)
df

,Loan ID,Segment,Region,Cost Centre,RM_ID,CAG_ID,GFCID,Facility Type,Facility ID,FRR,...,Maturity Date,Comp_Freq,Duration (Period),Base Rate,Rate Tenor,RFR,Client Spread (LIBOR),Client Spread (RFR),Currency,Exposure
0,25,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Commercial Loan,F25,5,...,2021-03-31,Monthly,12,LIBOR,1M,SONIA,1.5,1.5,GBP,760051
1,26,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Commercial Loan,F26,4,...,2023-03-31,Monthly,36,LIBOR,1M,SONIA,1.5,1.5,GBP,951917
2,27,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Commercial Loan,F27,4,...,2025-03-31,Monthly,60,LIBOR,1M,SONIA,1.5,1.5,GBP,1010716
3,28,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Equipment Loan,F28,5,...,2030-03-31,Monthly,120,LIBOR,1M,SONIA,1.5,1.5,GBP,1101088
4,29,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Equipment Loan,F29,5,...,2021-03-31,Quarterly,4,LIBOR,3M,SONIA,1.5,1.5,GBP,763730
5,30,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Equipment Loan,F30,5,...,2023-03-31,Quarterly,12,LIBOR,3M,SONIA,1.5,1.5,GBP,960718
6,31,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Commercial Loan,F31,4,...,2025-03-31,Quarterly,20,LIBOR,3M,SONIA,1.5,1.5,GBP,1024452
7,32,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Commercial Loan,F32,5,...,2030-03-31,Quarterly,40,LIBOR,3M,SONIA,1.5,1.5,GBP,1127710
8,33,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Equipment Loan,F33,6,...,2021-03-31,Six Months,2,LIBOR,6M,SONIA,1.5,1.5,GBP,764982
9,34,SEG-A,Europe,UK,RM_02,G02_LS,C04_JP,Commercial Loan,F34,4,...,2023-03-31,Six Months,6,LIBOR,6M,SONIA,1.5,1.5,GBP,959768


In [3]:
base_rate_calc={"Monthly":"1M","Quarterly":"3M","Six Months":"6M","Annually":"12M"}
time_duration_calc= {"Monthly":1,"Quarterly":3,"Six Months":6,"Annually":12}
effective_col_calc={"Monthly":12,"Quarterly":4,"Six Months":2,"Annually":1}
mapping_dict={"LIBOR":LIBOR_int,"SOFR":SOFR_int,"SONIA":SONIA_int,"TIIE":TIIE_int,"BSBY":BSBY_int}
date_interval={"Monthly":30,"Quarterly":90,"Six Months":180,"Annually":360}
exposure_cal={"Monthly":3,"Quarterly":1,"Six Months":0.5,"Annually":0.25}
currency_conv={"USD":1,"GBP":1.39,"MXN":0.05}

In [4]:
def numOfDays(date1, date2):
    "calculates number of days between two dates"
    return (date2-date1).days

def get_interest_rate(int_df,start_date,tenor):
    "returns interest rate for a particular date"
    interest_rate=int_df[int_df["Date"]==start_date][tenor].values
    return interest_rate

In [5]:
def fixed_rate_calculation(notional,start_date,maturity_date,comp_freq,duration,base_rate,rfr,spread_libor,spread_rfr,currency,df_cal_fixed,RFR_dict):
    "calculates all interest and impact calculation for fixed rate portfolio"
    
    start_date, maturity_date=pd.to_datetime(start_date), pd.to_datetime(maturity_date)
    tenor= base_rate_calc[comp_freq]
    start_date_orig=start_date
    
    ####### Base Rate interest calculation #########
    int_df=mapping_dict[base_rate]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["Effective LIBOR"].append((1+(interest_rate[0]+spread_libor)/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["LIBOR Interest"].append(round(float(df_cal_fixed["Effective LIBOR"][-1])*notional,2))
    df_cal_fixed["LIBOR Total Interest"].append((notional*(1+df_cal_fixed["Effective LIBOR"][-1])**duration)-notional)
    no_of_days= numOfDays(start_date_orig,maturity_date)
    df_cal_fixed["YIELD P.A. % (LIBOR+CLIENT SPREAD)"].append(((((df_cal_fixed["LIBOR Total Interest"][-1]/notional)+1)**(365/no_of_days))-1)*100)
    
    ############ RFR Interest Calculation ##########
    int_df=mapping_dict[rfr]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        #print(start_date)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["Effective RFR"].append((1+(interest_rate[0]+spread_rfr)/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["RFR Interest"].append(round(float(df_cal_fixed["Effective RFR"][-1])*notional,2))
    df_cal_fixed["RFR Total Interest"].append((notional*(1+df_cal_fixed["Effective RFR"][-1])**duration)-notional)
    no_of_days= numOfDays(start_date,maturity_date)
    df_cal_fixed["YIELD P.A. % (RFR+CLIENT SPREAD)"].append(((((df_cal_fixed["RFR Total Interest"][-1]/notional)+1)**(365/no_of_days))-1)*100)
    
    ########## RFR Calculated spread and interest calculation ########
    df_cal_fixed["NET NEUTRALITY YIELD PREMIUM"].append(df_cal_fixed["YIELD P.A. % (RFR+CLIENT SPREAD)"][-1]-df_cal_fixed["YIELD P.A. % (LIBOR+CLIENT SPREAD)"][-1])
    df_cal_fixed["NET NEUTRALITY YIELD PREMIUM (BP)"].append(((df_cal_fixed["NET NEUTRALITY YIELD PREMIUM"][-1]/100)*notional)/1000)
    df_cal_fixed["Impact (BP)"].append(((df_cal_fixed["RFR Total Interest"][-1]-df_cal_fixed["LIBOR Total Interest"][-1])/notional)*10000)
    df_cal_fixed["RFR Spread Cal"].append(((((df_cal_fixed["Effective LIBOR"][-1]+1)**effective_col_calc[comp_freq])-1)*100)-interest_rate[0])
    df_cal_fixed["Effective RFR Cal"].append((1+(interest_rate[0]+df_cal_fixed['RFR Spread Cal'][-1])/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["Interest Calc"].append(round(float(df_cal_fixed["Effective RFR Cal"][-1])*notional,2))
    df_cal_fixed["Total Interest Calc"].append((notional*(1+df_cal_fixed["Effective RFR Cal"][-1])**duration)-notional)
    
    
    RFR_dict[rfr].append(df_cal_fixed["RFR Spread Cal"][-1])
    return df_cal_fixed,RFR_dict 

def adjusted_RFR_interest_calculation(df_cal_fixed,RFR_dict,notional,rfr,start_date,duration,comp_freq,i):
    
    tenor= base_rate_calc[comp_freq]
    int_df=mapping_dict[rfr]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        #print(start_date)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["RFR Spread Adj"].append(round(np.mean(RFR_dict[rfr]),4))
    df_cal_fixed["Effective RFR Adj"].append((1+(interest_rate[0]+df_cal_fixed['RFR Spread Adj'][-1])/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["Interest Adj"].append(round(float(df_cal_fixed["Effective RFR Adj"][-1])*notional,2))
    df_cal_fixed["Total Interest Adj"].append((notional*(1+df_cal_fixed["Effective RFR Adj"][-1])**duration)-notional)
    df_cal_fixed["Impact"].append(df_cal_fixed["Total Interest Adj"][-1] - df_cal_fixed["Total Interest Calc"][i])
    df_cal_fixed["EMI RFR"].append((notional*df_cal_fixed["Effective LIBOR"][i]*(1+df_cal_fixed["Effective LIBOR"][i])**duration)/(((1+df_cal_fixed["Effective LIBOR"][i])**duration)-1))
    return df_cal_fixed                                   

In [6]:
inter_dict_base={}
inter_dict_RFR={}
inter_dict_RFR_adj={}

def floating_rate_calculation(loan_id,notional,start_date,maturity_date,comp_freq,duration,base_rate,rfr,spread_libor,spread_rfr,currency,df_cal_floating):
    ######## calculation of interest for floating rate portfolio #########
    
    inter_dict_base[loan_id]=[]
    inter_dict_RFR[loan_id]=[]
    inter_dict_RFR_adj[loan_id]=[]
    interest_monthly=[]
    start_date_orig=start_date
    for j in range(0,duration):
        ####Calculation of interest with Base Rate######

        start_date= start_date_orig+timedelta(days=int(j)*date_interval[comp_freq])
        int_df=mapping_dict[base_rate]
        tenor= base_rate_calc[comp_freq]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_libor)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_base[loan_id].append(notional*effective_rate)
        
        ####Calculation of interest with RFR Rate######
        int_df=mapping_dict[rfr]
        interest_rate=get_interest_rate(int_df,start_date,tenor)

        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_rfr)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_RFR[loan_id].append(notional*effective_rate)
    
    df_cal_floating["LIBOR Total Interest"].append(np.sum(inter_dict_base[loan_id]))
    df_cal_floating["RFR Total Interest"].append(np.sum(inter_dict_RFR[loan_id]))
    no_of_days=numOfDays(start_date_orig,maturity_date)
    df_cal_floating["YIELD P.A. % (LIBOR+CLIENT SPREAD)"].append((((df_cal_floating["LIBOR Total Interest"][-1]/notional)+1)**(365/no_of_days)-1)*100)
    df_cal_floating["YIELD P.A. % (RFR+CLIENT SPREAD)"].append((((df_cal_floating["RFR Total Interest"][-1]/notional)+1)**(365/no_of_days)-1)*100)
    df_cal_floating["NET NEUTRALITY YIELD PREMIUM"].append(df_cal_floating["YIELD P.A. % (RFR+CLIENT SPREAD)"][-1]-df_cal_floating["YIELD P.A. % (LIBOR+CLIENT SPREAD)"][-1])
    df_cal_floating["NET NEUTRALITY YIELD PREMIUM (BP)"].append(((df_cal_floating["NET NEUTRALITY YIELD PREMIUM"][-1]/100)*notional)/1000)

    ###weighted AVG Calculation###

    RFR_Weighted_AVG=(((((df_cal_floating["RFR Total Interest"][-1]/notional)/duration)+1)**effective_col_calc[comp_freq])-1)*100
    df_cal_floating["RFR Spread Adj"].append(((((1+((df_cal_floating["LIBOR Total Interest"][-1]/duration)/notional))**effective_col_calc[comp_freq])-1)*100)-RFR_Weighted_AVG)
    Adjusted_spred=df_cal_floating["RFR Spread Adj"][-1]
    
    ####Calculation of interest with RFR adjusted Rate######
    for j in range(0,duration):
        start_date= start_date_orig+timedelta(days=int(j)*date_interval[comp_freq])
        int_df=mapping_dict[rfr]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_rfr+Adjusted_spred)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_RFR_adj[loan_id].append(notional*effective_rate)
    
    df_cal_floating["Total Interest Adj"].append(np.sum(inter_dict_RFR_adj[loan_id]))
    return df_cal_floating

In [7]:
df_cal_fixed={"Effective LIBOR":[],"LIBOR Interest":[],"LIBOR Total Interest":[],"Effective RFR":[],"RFR Interest":[],"RFR Total Interest":[],"YIELD P.A. % (LIBOR+CLIENT SPREAD)":[],"YIELD P.A. % (RFR+CLIENT SPREAD)":[],"NET NEUTRALITY YIELD PREMIUM":[],"NET NEUTRALITY YIELD PREMIUM (BP)":[],"Impact (BP)":[],"RFR Spread Cal":[],"Effective RFR Cal":[],"Interest Calc":[],"Total Interest Calc":[],"RFR Spread Adj":[],"Effective RFR Adj":[],"Interest Adj":[],"Total Interest Adj":[],"Impact":[],"EMI RFR":[]}
df_cal_floating={"LIBOR Total Interest":[],"RFR Total Interest":[],"YIELD P.A. % (LIBOR+CLIENT SPREAD)":[],"YIELD P.A. % (RFR+CLIENT SPREAD)":[],"NET NEUTRALITY YIELD PREMIUM":[],"NET NEUTRALITY YIELD PREMIUM (BP)":[],"RFR Spread Adj":[],"Total Interest Adj":[]}

####select loan for a customer##### 
Customer= "C04_JP"
df=df[df["GFCID"]==Customer]
df.index=pd.RangeIndex(len(df.index))

#####calculate interests for all loans with yield calculation###### 
rfr=list(df["RFR"].unique())
RFR_dict={}
for item in rfr:
    RFR_dict[item]=[]
for i in range(len(df)):
    loan_id,notional,start_date,maturity_date=df.loc[i,"Loan ID"],df.loc[i,"Notional Amt"],df.loc[i,"Start Date"],df.loc[i,"Maturity Date"]
    comp_freq,duration,nature=df.loc[i,"Comp_Freq"],df.loc[i,"Duration (Period)"],df.loc[i,"Loan Nature"]
    base_rate,rfr,spread_libor,spread_rfr,currency=df.loc[i,"Base Rate"],df.loc[i,"RFR"],df.loc[i,"Client Spread (LIBOR)"],df.loc[i,"Client Spread (RFR)"],df.loc[i,"Currency"]
    if nature=="Fixed":
        df_cal_fixed,RFR_dict=fixed_rate_calculation(notional,start_date,maturity_date,comp_freq,duration,base_rate,rfr,spread_libor,spread_rfr,currency,df_cal_fixed,RFR_dict)
    if nature=="Floating":
        df_cal_floating = floating_rate_calculation(loan_id,notional,start_date,maturity_date,comp_freq,duration,base_rate,rfr,spread_libor,spread_rfr,currency,df_cal_floating)
for i in range(len(df)):
    notional,start_date,rfr,duration,comp_freq,nature= df.loc[i,"Notional Amt"],df.loc[i,"Start Date"],df.loc[i,"RFR"],df.loc[i,"Duration (Period)"],df.loc[i,"Comp_Freq"],df.loc[i,"Loan Nature"]
    if nature=="Fixed":
        df_cal_fixed=adjusted_RFR_interest_calculation(df_cal_fixed,RFR_dict,notional,rfr,start_date,duration,comp_freq,i)
int_df_fixed=pd.DataFrame(df_cal_fixed)
int_df_floating= pd.DataFrame(df_cal_floating)
int_df_fixed

,Effective LIBOR,LIBOR Interest,LIBOR Total Interest,Effective RFR,RFR Interest,RFR Total Interest,YIELD P.A. % (LIBOR+CLIENT SPREAD),YIELD P.A. % (RFR+CLIENT SPREAD),NET NEUTRALITY YIELD PREMIUM,NET NEUTRALITY YIELD PREMIUM (BP),...,RFR Spread Cal,Effective RFR Cal,Interest Calc,Total Interest Calc,RFR Spread Adj,Effective RFR Adj,Interest Adj,Total Interest Adj,Impact,EMI RFR
0,0.002054,2054.04,24928.800000,0.001300,1299.88,1.571060e+04,2.492880,1.571060,-0.921820,-9.218200,...,2.42182,0.002054,2054.04,24928.800000,2.5860,0.002188,2187.70,26570.600000,1641.800000,84450.121343
1,0.002054,2054.04,76666.227088,0.001300,1299.88,4.787615e+04,2.492880,1.571060,-0.921820,-9.218200,...,2.42182,0.002054,2054.04,76666.227088,2.5860,0.002188,2187.70,81848.549111,5182.322023,28845.957120
2,0.002054,2054.04,131015.310090,0.001300,1299.88,8.106031e+04,2.491498,1.570193,-0.921305,-9.213050,...,2.42182,0.002054,2054.04,131015.310090,2.5860,0.002188,2187.70,140103.060826,9087.750736,17731.863900
3,0.002054,2054.04,279195.631658,0.001300,1299.88,1.686914e+05,2.491498,1.570193,-0.921305,-9.213050,...,2.42182,0.002054,2054.04,279195.631658,2.5860,0.002188,2187.70,299834.989305,20639.357647,9411.011480
4,0.007296,7296.01,29505.000000,0.003911,3910.50,1.573400e+04,2.950500,1.573400,-1.377100,-13.771000,...,2.87710,0.007296,7296.01,29505.000000,2.5860,0.006583,6583.21,26594.000000,-2911.000000,254576.582550
5,0.007296,7296.01,91152.320506,0.003911,3910.50,4.794857e+04,2.950500,1.573400,-1.377100,-13.771000,...,2.87710,0.007296,7296.01,91152.320506,2.5860,0.006583,6583.21,81922.530871,-9229.789635,87338.003992
6,0.007296,7296.01,156491.116163,0.003911,3910.50,8.118485e+04,2.948861,1.572532,-1.376329,-13.763290,...,2.87710,0.007296,7296.01,156491.116163,2.5860,0.006583,6583.21,140233.006238,-16258.109925,53918.552405
7,0.007296,7296.01,337471.701764,0.003911,3910.50,1.689607e+05,2.948861,1.572532,-1.376329,-13.763290,...,2.87710,0.007296,7296.01,337471.701764,2.5860,0.006583,6583.21,300131.308514,-37340.393250,28915.642817
8,0.013288,13287.96,26752.500000,0.007858,7857.78,1.577730e+04,2.675250,1.577730,-1.097520,-10.975200,...,2.59752,0.013288,13287.96,26752.500000,2.5860,0.013231,13231.12,26637.300000,-115.200000,509987.899323
9,0.013288,13287.96,82423.735433,0.007858,7857.78,4.808260e+04,2.675250,1.577730,-1.097520,-10.975200,...,2.59752,0.013288,13287.96,82423.735433,2.5860,0.013231,13231.12,82059.437637,-364.297796,174503.237795


In [8]:
int_df_floating

,LIBOR Total Interest,RFR Total Interest,YIELD P.A. % (LIBOR+CLIENT SPREAD),YIELD P.A. % (RFR+CLIENT SPREAD),NET NEUTRALITY YIELD PREMIUM,NET NEUTRALITY YIELD PREMIUM (BP),RFR Spread Adj,Total Interest Adj


In [9]:
df_fixed=df[df["Loan Nature"]=="Fixed"]
df_fixed.index=pd.RangeIndex(len(df_fixed))
df_agg_yield_cal_fi=int_df_fixed.loc[:, :'NET NEUTRALITY YIELD PREMIUM (BP)']
df_agg_yield_cal_fi=pd.concat([df_fixed,df_agg_yield_cal_fi],axis=1)
df_fi_copy=copy.deepcopy(df_agg_yield_cal_fi)

df_floating=df[df["Loan Nature"]=="Floating"]
df_floating.index=pd.RangeIndex(len(df_floating))
df_agg_yield_cal_fl=int_df_floating.loc[:, :'NET NEUTRALITY YIELD PREMIUM (BP)']
df_agg_yield_cal_fl=pd.concat([df_floating,df_agg_yield_cal_fl],axis=1)
df_fl_copy=copy.deepcopy(df_agg_yield_cal_fl)

#print(df_agg_yield_cal_fl)
def convert_currency(df_agg_yield_cal,currency_conv):
    
    
    for i in range(len(df_agg_yield_cal)):
        start_date,maturity_date=pd.to_datetime(df_agg_yield_cal.loc[i,"Start Date"]),pd.to_datetime(df_agg_yield_cal.loc[i,"Maturity Date"])
        no_of_days=numOfDays(start_date,maturity_date)
        df_agg_yield_cal.loc[i,"Notional Amt USD Eq"]=df_agg_yield_cal.loc[i,"Notional Amt"]*currency_conv[df_agg_yield_cal.loc[i,"Currency"]]
        df_agg_yield_cal.loc[i,"Exposure USD Equivalent"]=df_agg_yield_cal.loc[i,"Exposure"]*currency_conv[df_agg_yield_cal.loc[i,"Currency"]]
        df_agg_yield_cal.loc[i,"LIBOR Interest USD Eq"]=df_agg_yield_cal.loc[i,"LIBOR Total Interest"]*currency_conv[df_agg_yield_cal.loc[i,"Currency"]]
        df_agg_yield_cal.loc[i,"RFR Interest USD Eq"]=df_agg_yield_cal.loc[i,"RFR Total Interest"]*currency_conv[df_agg_yield_cal.loc[i,"Currency"]]
        df_agg_yield_cal.loc[i,"Yield USD Eq (LIBOR)"]=((((df_agg_yield_cal.loc[i,"LIBOR Interest USD Eq"]/df_agg_yield_cal.loc[i,"Notional Amt USD Eq"])+1)**(365/no_of_days))-1)
        df_agg_yield_cal.loc[i,"Yield USD Eq (RFR)"]=((((df_agg_yield_cal.loc[i,"RFR Interest USD Eq"]/df_agg_yield_cal.loc[i,"Notional Amt USD Eq"])+1)**(365/no_of_days))-1)
        df_agg_yield_cal.loc[i,"Net Neutrality"]=df_agg_yield_cal.loc[i,"Yield USD Eq (RFR)"]-df_agg_yield_cal.loc[i,"Yield USD Eq (LIBOR)"]

    return df_agg_yield_cal

def yield_aggregation_currency_conv(df_agg_yield_cal):
    
    Total_exposure=np.sum(df_agg_yield_cal["Exposure"])
    Total_conv_exposure=np.sum(df_agg_yield_cal["Exposure USD Equivalent"])
    #print(Total_conv_exposure)
    for i in range(len(df_agg_yield_cal)):
        df_agg_yield_cal.loc[i,"Weights"]=df_agg_yield_cal.loc[i,"Exposure USD Equivalent"]/Total_conv_exposure
        df_agg_yield_cal.loc[i,"Yield WA LIBOR"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"Yield USD Eq (LIBOR)"]
        df_agg_yield_cal.loc[i,"Yield WA RFR"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"Yield USD Eq (RFR)"]
        df_agg_yield_cal.loc[i,"Net Neutrality Yield"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"Net Neutrality"]
        
    #print(df_agg_yield_cal)

    Yield_stat={"Total Exposure":Total_exposure,"Total Exposure USD Equivalent":Total_conv_exposure,"WA Yield (LIBOR)":np.sum(df_agg_yield_cal["Yield WA LIBOR"]),"WA Yield (RFR)":np.sum(df_agg_yield_cal["Yield WA RFR"]),"WA Yield Difference":np.sum(df_agg_yield_cal["Net Neutrality Yield"])}
    yield_df=pd.DataFrame(Yield_stat,index=[0])
    #print(yield_df)
    return df_agg_yield_cal,yield_df

def yield_aggregation(df_agg_yield_cal):
    
    df_agg_yield_cal.index=pd.RangeIndex(len(df_agg_yield_cal))
    Total_exposure=np.sum(df_agg_yield_cal["Exposure"])
    #print(Total_exposure)
    for i in range(len(df_agg_yield_cal)):
        df_agg_yield_cal.loc[i,"Weights"]=df_agg_yield_cal.loc[i,"Exposure"]/Total_exposure
        df_agg_yield_cal.loc[i,"Yield WA LIBOR"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"YIELD P.A. % (LIBOR+CLIENT SPREAD)"]
        df_agg_yield_cal.loc[i,"Yield WA RFR"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"YIELD P.A. % (RFR+CLIENT SPREAD)"]
        df_agg_yield_cal.loc[i,"Net Neutrality Yield"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"NET NEUTRALITY YIELD PREMIUM"]
        df_agg_yield_cal.loc[i,"Net Neutrality Yield (BP)"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"NET NEUTRALITY YIELD PREMIUM (BP)"]
    #print(df_agg_yield_cal)

    Yield_stat={"Total Exposure":Total_exposure,"WA Yield (LIBOR)":np.sum(df_agg_yield_cal["Yield WA LIBOR"]),"WA Yield (RFR)":np.sum(df_agg_yield_cal["Yield WA RFR"]),"WA Yield Difference":np.sum(df_agg_yield_cal["Net Neutrality Yield"]),"WA Yield Difference (BP)":np.sum(df_agg_yield_cal["Net Neutrality Yield (BP)"])}
    yield_df=pd.DataFrame(Yield_stat,index=[0])
    #print(yield_df)
    return df_agg_yield_cal,yield_df





In [10]:
#####currency conversion and yield aggregation######
try:
    df_agg_yield_cal_fi=convert_currency(df_agg_yield_cal_fi,currency_conv)
    df_agg_yield_cal_fi,fi_summary_df=yield_aggregation_currency_conv(df_agg_yield_cal_fi)
    print(df_agg_yield_cal_fi)
    print(fi_summary_df)
except:
    pass


    Loan ID Segment  Region Cost Centre  RM_ID  CAG_ID   GFCID  \
0        25   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
1        26   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
2        27   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
3        28   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
4        29   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
5        30   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
6        31   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
7        32   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
8        33   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
9        34   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
10       35   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
11       36   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
12       37   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
13       38   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
14       3

In [11]:

try:
    df_agg_yield_cal_fl=convert_currency(df_agg_yield_cal_fl,currency_conv)
    df_agg_yield_cal_fl,fl_summary_df=yield_aggregation_currency_conv(df_agg_yield_cal_fl)
    print(df_agg_yield_cal_fl)
    print(fl_summary_df)
except:
    pass

In [12]:

#####for separating currency based lone and calculation of yield aggregation for all parts####
try:

    dict_of_currency_fi = {k: v for k, v in df_fi_copy.groupby('Currency')}
    summary_dict_fi=dict.fromkeys(dict_of_currency_fi)
    #print(summary_dict_fi)
    for key, values in dict_of_currency_fi.items():
        #print(key)
        dict_of_currency_fi[key],summary_df=yield_aggregation(values)
        #print(summary_df)
        summary_dict_fi[key]=summary_df
        
except:
    pass
print(dict_of_currency_fi)
print(summary_dict_fi)        
    


{'GBP':     Loan ID Segment  Region Cost Centre  RM_ID  CAG_ID   GFCID  \
0        25   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
1        26   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
2        27   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
3        28   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
4        29   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
5        30   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
6        31   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
7        32   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
8        33   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
9        34   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
10       35   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
11       36   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
12       37   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
13       38   SEG-A  Europe          UK  RM_02  G02_LS  C04_JP   
14

In [13]:
try:

    dict_of_currency_fl = {k: v for k, v in df_fl_copy.groupby('Currency')}
    summary_dict_fl=dict.fromkeys(dict_of_currency_fl)
    print(summary_dict_fl)
    for key, values in dict_of_currency_fl.items():
        print(key)
        dict_of_currency_fl[key],summary_df=yield_aggregation(values)
        #print(summary_df)
        summary_dict_fl[key]=summary_df
        
except:
    pass
print(dict_of_currency_fl)
print(summary_dict_fl) 

{}
{}
{}
